In [199]:
# 교차 검증
# 부족한 데이터 셋 및 특성 데이터에 과대적합되는 문제를 해결하기 위한 방안
# 학습 데이터셋을 일정 크기의 데이터로 n개 분리 후 1/n은 검증용, 나머지는 학습용으로 사용

In [200]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

perchDF=pd.read_csv("perch3.csv")

In [201]:
fold_5=KFold(3)
for index, (train_index, test_index) in enumerate(fold_5.split(perchDF)):
    print(index)
    print(train_index.shape, test_index.shape)

0
(37,) (19,)
1
(37,) (19,)
2
(38,) (18,)


In [202]:
# 타겟이 분류인 경우 

df=pd.read_csv("../data/iris.csv", names=["sepal_length","sepal_width","petal_length","petal_width","Species"])
k_fold=KFold(n_splits=5, shuffle=True)
ret=k_fold.split(df["Species"])
train_data=[]
test_data=[]
for train_index, test_index in ret:
    train_data.append(df[["Species"]].iloc[train_index])
    test_data.append(df[["Species"]].iloc[test_index])
data=[]
for i in range(len(train_data)):
    data.append(train_data[i].value_counts().values)
ratio=pd.DataFrame(np.array(data).T, index=df.Species.unique())
ratio

,0,1,2,3,4
Iris-setosa,41,42,42,43,43
Iris-versicolor,41,40,40,43,39
Iris-virginica,38,38,38,34,38


In [203]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
y=le.fit_transform(df.Species)
k_fold=StratifiedKFold(n_splits=5)

ret=k_fold.split(df["Species"],y)
train_data=[]
test_data=[]
for train_index, test_index in ret:
    train_data.append(df[["Species"]].iloc[train_index])
    test_data.append(df[["Species"]].iloc[test_index])
data=[]
for i in range(len(train_data)):
    data.append(train_data[i].value_counts().values)
ratio=pd.DataFrame(np.array(data).T, index=df.Species.unique())
ratio

,0,1,2,3,4
Iris-setosa,40,40,40,40,40
Iris-versicolor,40,40,40,40,40
Iris-virginica,40,40,40,40,40


In [204]:
from sklearn.linear_model import LogisticRegression

df=pd.read_csv("../data/iris.csv", names=["sepal_length","sepal_width","petal_length","petal_width","Species"])

k_fold=KFold(n_splits=4, shuffle=True, random_state=123) # KFold 객체 생성
train_score_list=[] # 각 분할된 데이터의 훈련성적
test_score_list=[] # 각 분할된 데이터의 테스트성적
for idx, (train, test) in enumerate(k_fold.split(df["Species"])) :
    
    # 0. 학습용, 테스트용 인덱스 반환
    train_idx=train.tolist()
    test_idx=test.tolist()
    
    # 1. 인덱스에 해당하는 데이터 셋 추출
    trainDF=df.iloc[train_idx]
    testDF=df.iloc[test_idx]
    if False : 
        print(trainDF.Species.value_counts()/trainDF.shape[0])
        print(testDF.Species.value_counts()/testDF.shape[0])
    
    # 2. 분류 모델 학습
    x_train=trainDF[trainDF.columns[:-1]]
    y_train=trainDF[trainDF.columns[-1]]
    x_test=testDF[testDF.columns[:-1]]
    y_test=testDF[testDF.columns[-1]]
    
    log_model=LogisticRegression(max_iter=10000)
    log_model.fit(x_train, y_train)
    
    # 3. 훈련 및 검증용
    train_score=log_model.score(x_train, y_train)
    test_score=log_model.score(x_test, y_test)
    train_score_list.append(train_score)
    test_score_list.append(test_score)
    
# 각 데이터 셋들의 성적들을 평균화함 
print(np.mean(train_score_list), np.mean(test_score_list))

0.9755254424778761 0.966394025604552
